<!-- # Regresión y Selección de Modelos

## Aplicaciones en Ingeniería Electrónica

Este notebook contiene ejemplos prácticos y aplicaciones de regresión y selección de modelos en el contexto de ingeniería electrónica.

<a id="contenido"></a>

### Temas Cubiertos:

[**Ejemplos de Aplicaciones Prácticas**](#x)

1. [**Calibración de Sensores**](#calibracion-sensores)
2. [**Predicción de Fallas en Circuitos**](#prediccion-fallas)
3. [**Optimización de Parámetros de Fabricación**](#optimizacion-parametros)
4. [**Selección de Modelos y Validación Cruzada**](#seleccion-modelos)
5. [**Control de Procesos Industriales**](#control-procesos)

[**Resumen y Conclusiones**](#resumen) -->

<a id="contenido"></a>


# Regresión y Selección de Modelos en Machine Learning


## Aplicaciones Prácticas

1. **[Calibración de Sensores de Temperatura](#calibracion-sensores)**
2. **[Optimización de Parámetros de Fabricación de Resistencias](#optimizacion-parametros)**
3. **[Selección de Modelos y Validación Cruzada](#seleccion-modelos)**

---

### Objetivos de Aprendizaje

- Aplicar modelos de regresión lineal a problemas reales de ingeniería
- Comparar diferentes algoritmos: Linear, Ridge, Lasso, Random Forest
- Implementar técnicas de validación cruzada y optimización de hiperparámetros
- Evaluar modelos usando métricas apropiadas (RMSE, R², MAE)
- Interpretar resultados y seleccionar el mejor modelo

---

### Importar librerías necesarias

In [ ]:
# Importar librerías necesarias
import numpy as np               # Operaciones numéricas y arrays
import pandas as pd              # Manipulación de datos tabulares
import matplotlib.pyplot as plt  # Visualización de datos
import seaborn as sns            # Visualización estadística avanzada

# Modelos de Machine Learning
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
# StandardScaler: Estandarización, PolynomialFeatures: Crear términos polinomiales
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
# train_test_split: Dividir datos, cross_val_score: Validación cruzada, GridSearchCV: Optimización
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
# Métricas para evaluación de modelos
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, classification_report, confusion_matrix
# Modelos de ensemble (combinación de múltiples modelos)
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

# Suprimir advertencias para visualización limpia
import warnings
warnings.filterwarnings('ignore')

# Configurar estilo de gráficos
plt.style.use('seaborn-v0_8')   # Estilo visual moderno
sns.set_palette("husl")         # Paleta de colores armoniosa

print("Librerías importadas correctamente")

<a id="calibracion-sensores"></a>

## 1. Calibración de Sensores de Temperatura [⬆](#contenido)

### Problema:
Un sensor de temperatura tiene desgaste y no linealidad. Necesitamos crear un modelo de calibración que convierta la lectura del sensor (voltaje) a temperatura real.

### Datos:
- Temperatura real (°C): 5, 25, 50, 75, 100
- Lectura del sensor (V): 0.1, 1.2, 2.4, 3.6, 4.8


In [ ]:
# Crear datos de calibración del sensor

# Puntos de referencia conocidos (medidos con termómetro de precisión)
temperatura_real = np.array([5, 25, 50, 75, 100])     # °C, temperatura real medida
lectura_sensor = np.array([0.1, 1.2, 2.4, 3.6, 4.8])  #  V, voltaje de salida del sensor

# Crear DataFrame para organizar los datos de manera tabular
df_sensor = pd.DataFrame({
    'lectura_sensor': lectura_sensor,
    'temperatura_real': temperatura_real,
})

print("Datos del sensor:")
print(df_sensor)

# Visualizar la relación
plt.figure(figsize=(10, 6))
plt.scatter(lectura_sensor, temperatura_real, s=100, alpha=0.7, color='blue')
plt.xlabel('Lectura del Sensor (V)')
plt.ylabel('Temperatura Real (°C)')
plt.title('Calibración de Sensor de Temperatura')
plt.grid(True, alpha=0.3)
plt.show()


In [ ]:
# Ajustar modelo de regresión lineal

# Reshape(-1, 1): Convierte el array 1D en matriz columna (necesario para sklearn)
X_sensor = lectura_sensor.reshape(-1, 1)
y_sensor = temperatura_real

# Modelo lineal simple
modelo_lineal = LinearRegression()
# fit(): Entrena el modelo encontrando los coeficientes que minimizan el error cuadrático
modelo_lineal.fit(X_sensor, y_sensor)

# Predecir temperaturas
# predict(): Aplica la ecuación T = intercept + coef * V a los datos de entrada
y_pred_lineal = modelo_lineal.predict(X_sensor)

# Calcular métricas

# MSE: Error cuadrático medio, penaliza errores grandes
mse_lineal = mean_squared_error(y_sensor, y_pred_lineal)

# RMSE: Raíz del MSE, en las mismas unidades que la variable objetivo
rmse_lineal = np.sqrt(mse_lineal)

# R²: Coeficiente de determinación, mide qué tan bien el modelo explica la variabilidad (0-1)
r2_lineal = r2_score(y_sensor, y_pred_lineal)

print(f"Modelo Lineal:")
print(f"Ecuación: T = {modelo_lineal.intercept_:.2f} + {modelo_lineal.coef_[0]:.2f} * V")
print(f"RMSE: {rmse_lineal:.2f} °C")
print(f"R²: {r2_lineal:.4f}")

# Visualizar resultados
plt.figure(figsize=(12, 5))

# Subgráfico 1: Ajuste del modelo
plt.subplot(1, 2, 1)

plt.scatter(lectura_sensor, temperatura_real, s=100, alpha=0.7, color='blue', label='Datos reales')
    # s: tamaño de puntos, alpha: transparencia

# Línea de regresión (predicciones del modelo)
plt.plot(lectura_sensor, y_pred_lineal, 'r-', linewidth=2, label='Modelo lineal')

plt.xlabel('Lectura del Sensor (V)')
plt.ylabel('Temperatura Real (°C)')
plt.title('Calibración Lineal')
plt.legend()
plt.grid(True, alpha=0.3)

# Subgráfico 2: Análisis de residuos
plt.subplot(1, 2, 2)

# Residuos: Diferencia entre valor real y predicción (e_i = y_i - ŷ_i)
residuos = temperatura_real - y_pred_lineal
plt.scatter(y_pred_lineal, residuos, s=100, alpha=0.7, color='green')

# Línea horizontal en y=0 (residuos ideales deben estar alrededor de 0)
plt.axhline(y=0, color='r', linestyle='--')

plt.xlabel('Temperatura Predicha (°C)')
plt.ylabel('Residuos (°C)')
plt.title('Análisis de Residuos')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

<a id="optimizacion-parametros"></a>

## 2. Optimización de Parámetros de Fabricación de Resistencias [⬆](#contenido)

### Problema:
Optimizar los parámetros de fabricación para obtener resistencias con valores específicos.

### Variables:
- **Temperatura de cocción** (°C)
- **Tiempo de cocción** (minutos)
- **Espesor del material** (μm)
- **Concentración de dopante** (%)
- **Objetivo**: Resistencia (Ω)


In [ ]:
# Generar datos de fabricación de resistencias

np.random.seed(127)
n_resistencias = 150

# Parámetros de fabricación

# uniform(min, max, n): Distribución uniforme (todos los valores igualmente probables)
temp_coccion = np.random.uniform(800, 1200, n_resistencias)          # °C, rango de cocción típico
tiempo_coccion = np.random.uniform(30, 120, n_resistencias)          # min
espesor = np.random.uniform(10, 50, n_resistencias)                  # μm, espesor de película
concentracion_dopante = np.random.uniform(0.1, 5.0, n_resistencias)  # %, dopaje del material

# Modelo físico simplificado para resistencia
# Basado en ley de Ohm: R = ρ * L / A, donde ρ depende de temperatura y dopante
# exp(): Función exponencial, modela efecto no lineal de temperatura
resistividad_base = 0.1                             # Ω·μm, resistividad base del material
factor_temp = np.exp(-(temp_coccion - 1000) / 200)  # Efecto de temperatura (Arrhenius)
factor_dopante = 1 / (1 + concentracion_dopante)    # Dopante reduce resistencia
factor_espesor = 1 / espesor                        # Menor espesor = mayor resistencia

# Calcular resistencia con modelo físico más ruido gaussiano (simula imperfecciones)
resistencia = (resistividad_base * factor_temp * factor_dopante * factor_espesor *
               (1 + 0.1 * np.random.normal(0, 1, n_resistencias)))  # 10% de ruido

# Crear DataFrame
df_resistencias = pd.DataFrame({
    'temp_coccion':          temp_coccion,
    'tiempo_coccion':        tiempo_coccion,
    'espesor':               espesor,
    'concentracion_dopante': concentracion_dopante,
    'resistencia':           resistencia
})

print("Datos de fabricación de resistencias:")

In [ ]:
print(f"\nMuestras:")
df_resistencias.head()

In [ ]:
print(f"\nEstadísticas descriptivas:")
df_resistencias.describe()

In [ ]:
# Análisis exploratorio de datos de resistencias

plt.figure(figsize=(15, 12))

# Relaciones entre variables independientes y la variable objetivo (resistencia)
variables = ['temp_coccion', 'tiempo_coccion', 'espesor', 'concentracion_dopante']
for i, var in enumerate(variables):
    plt.subplot(2, 2, i+1)  # Grid 2x2 para visualizar todas las variables

    # Gráfico de dispersión para identificar relaciones lineales o no lineales
    plt.scatter(df_resistencias[var], df_resistencias['resistencia'], alpha=0.6)

    plt.xlabel(var)
    plt.ylabel('Resistencia (Ω)')
    plt.title(f'Resistencia vs {var}')
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 8))

# Matriz de correlación: Identificar multicolinealidad entre variables
correlation_matrix = df_resistencias.corr()
    # Valores cercanos a 1/-1: Fuerte correlación positiva/negativa
    # Valores cercanos a 0: No hay correlación lineal

sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,
            square=True, fmt='.2f')
plt.title('Matriz de Correlación - Resistencias')
plt.show()


In [ ]:
# Entrenar y comparar diferentes modelos de regresión

# Separar variables predictoras (X) y variable objetivo (y)
X_resistencias = df_resistencias[['temp_coccion', 'tiempo_coccion', 'espesor', 'concentracion_dopante']]
y_resistencias = df_resistencias['resistencia']

# Dividir datos: 70% entrenamiento, 30% prueba
# test_size=0.3: 30% para test, random_state: Reproducibilidad
X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(X_resistencias, y_resistencias,
                                                           test_size=0.3, random_state=42)

# Estandarizar: Transformar datos a media=0 y desviación=1
# Crítico para Ridge, Lasso y otros modelos sensibles a la escala
scaler_r = StandardScaler()
X_train_r_scaled = scaler_r.fit_transform(X_train_r)    # Ajustar y transformar train
X_test_r_scaled = scaler_r.transform(X_test_r)          # Solo transformar test (no ajustar)

# Modelos a comparar
modelos = {
    'Regresión Lineal': LinearRegression(),             # Regresión sin regularización
    'Ridge (α=0.1)':    Ridge(alpha=0.1),               # Regularización L2 leve (penaliza coef. grandes)
    'Ridge (α=1.0)':    Ridge(alpha=1.0),               # Regularización L2 moderada
    'Lasso (α=0.01)':   Lasso(alpha=0.01),              # Regularización L1 leve (puede hacer coef. = 0)
    'Lasso (α=0.1)':    Lasso(alpha=0.1),               # Regularización L1 moderada
    'Random Forest':    RandomForestRegressor(n_estimators=100, random_state=42)  # Ensemble de 100 árboles
}

resultados = {}

print("Comparación de Modelos de Regresión:")
print("=" * 50)

for nombre_modelo, modelo in modelos.items():
    # Entrenar modelo según su tipo
    if nombre_modelo == 'Random Forest':
        # Random Forest: No necesita estandarización (basado en árboles)
        modelo.fit(X_train_r, y_train_r)
        y_pred = modelo.predict(X_test_r)
    else:
        # Modelos lineales: Requieren datos estandarizados para convergencia óptima
        modelo.fit(X_train_r_scaled, y_train_r)
        y_pred = modelo.predict(X_test_r_scaled)

    # Calcular métricas de regresión
    mse = mean_squared_error(y_test_r, y_pred)   # Error cuadrático medio
    rmse = np.sqrt(mse)                          # Raíz del MSE (mismas unidades que objetivo)
    mae = mean_absolute_error(y_test_r, y_pred)  # Error absoluto medio (robusto a outliers)
    r2 = r2_score(y_test_r, y_pred)              # R² (proporción de varianza explicada, 0-1)

    resultados[nombre_modelo] = {'RMSE': rmse, 'MAE': mae, 'R²': r2}
    # print(f"{nombre_modelo:20} | RMSE: {rmse:.4f} | MAE: {mae:.4f} | R²: {r2: .4f} |")

# Crear DataFrame de resultados
df_resultados = pd.DataFrame(resultados).T
print(df_resultados.round(4))


In [ ]:
# Visualizar comparación de modelos
plt.figure(figsize=(15, 5))

# Gráfico de barras para RMSE (menor es mejor)
plt.subplot(1, 3, 1)
df_resultados['RMSE'].plot(kind='bar', color='skyblue')
plt.title('Comparación de RMSE')
plt.ylabel('RMSE')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# Gráfico de barras para R² (más cercano a 1 es mejor)
plt.subplot(1, 3, 2)
df_resultados['R²'].plot(kind='bar', color='lightgreen')
plt.title('Comparación de R²')
plt.ylabel('R²')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# Gráfico de dispersión: Predicho vs Real (mejor modelo)
# idxmax(): Retorna el índice del valor máximo (mejor R²)
mejor_modelo = df_resultados['R²'].idxmax()
# Reentrenar el mejor modelo para generar predicciones
if mejor_modelo == 'Random Forest':
    modelo_mejor = RandomForestRegressor(n_estimators=100, random_state=42)
    modelo_mejor.fit(X_train_r, y_train_r)
    y_pred_mejor = modelo_mejor.predict(X_test_r)
else:
    modelo_mejor = Ridge(alpha=0.1)
    modelo_mejor.fit(X_train_r_scaled, y_train_r)
    y_pred_mejor = modelo_mejor.predict(X_test_r_scaled)

plt.subplot(1, 3, 3)
plt.scatter(y_test_r, y_pred_mejor, alpha=0.6, color='red')
# Línea diagonal perfecta (y=x): Si predicción = real, punto cae en esta línea
plt.plot([y_test_r.min(), y_test_r.max()], [y_test_r.min(), y_test_r.max()], 'k--', lw=2)
plt.xlabel('Resistencia Real (Ω)')
plt.ylabel('Resistencia Predicha (Ω)')
plt.title(f'Predicción vs Real - {mejor_modelo}')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nMejor modelo: {mejor_modelo}")
print(f"R² = {df_resultados.loc[mejor_modelo, 'R²']:.4f}")
print(f"RMSE = {df_resultados.loc[mejor_modelo, 'RMSE']:.4f}")


<a id="seleccion-modelos"></a>

## 3. Selección de Modelos y Validación Cruzada [⬆](#contenido)

### Problema:
Implementar técnicas de selección de modelos y validación cruzada para encontrar el mejor modelo.


In [ ]:
# Validación cruzada para selección de hiperparámetros

# Definir rangos de hiperparámetros para Ridge
# alpha: Parámetro de regularización (valores más altos = más regularización)
param_grid_ridge = {
    'alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]
}

# Grid Search con validación cruzada
# cv=5: Divide datos en 5 partes, entrena con 4 y valida con 1 (repite 5 veces)
# scoring: Métrica a optimizar (negativo porque sklearn maximiza)
ridge_cv = GridSearchCV(
    Ridge(),                           # Modelo a optimizar
    param_grid_ridge,                  # Grade de parámetros
    cv=5,                              # 5-fold cross-validation
    scoring='neg_mean_squared_error'   # Métrica a optimizar
)

# Prueba todas las combinaciones de hiperparámetros y selecciona la mejor
ridge_cv.fit(X_train_r_scaled, y_train_r)

print("Mejores parámetros para Ridge:")

# best_params_: Diccionario con los hiperparámetros óptimos encontrados
print(ridge_cv.best_params_)

# best_score_: Mejor score promedio obtenido en validación cruzada
print(f"Mejor score (negativo MSE): {ridge_cv.best_score_:.4f}")

# Comparar modelos con validación cruzada usando mejores hiperparámetros
modelos_cv = {
    'Linear Regression': LinearRegression(),
    'Ridge (CV)':        Ridge(alpha=ridge_cv.best_params_['alpha']),  # Usar alpha óptimo
    'Lasso':             Lasso(alpha=0.01),
    'Random Forest':     RandomForestRegressor(n_estimators=100, random_state=42)
}

print("\nValidación Cruzada (5-fold):")
print("=" * 40)

cv_scores = {}
for nombre, modelo in modelos_cv.items():
    # cross_val_score: Evalúa el modelo usando validación cruzada k-fold
    # Divide datos en k partes, entrena k veces y promedia resultados
    if nombre == 'Random Forest':
        # Random Forest no requiere estandarización (invariante a escala)
        scores = cross_val_score(modelo, X_train_r, y_train_r, cv=5, scoring='neg_mean_squared_error')
    else:
        # Modelos lineales requieren datos estandarizados
        scores = cross_val_score(modelo, X_train_r_scaled, y_train_r, cv=5, scoring='neg_mean_squared_error')

    # Almacenar resultados de validación cruzada
    cv_scores[nombre] = {
        'mean_score': scores.mean(),        # Promedio de scores entre los k folds
        'std_score': scores.std(),          # Desviación estándar (mide variabilidad)
        'rmse_cv': np.sqrt(-scores.mean())  # Convertir MSE negativo a RMSE positivo
    }

    # Mostrar RMSE ± desviación (menor RMSE y menor desviación = mejor modelo)
    print(f"{nombre:20} | RMSE: {np.sqrt(-scores.mean()):.4f} ± {np.sqrt(scores.std()):.4f}")

# Visualizar resultados de validación cruzada
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
nombres = list(cv_scores.keys())
rmse_values = [cv_scores[n]['rmse_cv'] for n in nombres]
std_values = [cv_scores[n]['std_score'] for n in nombres]

# Gráfico de barras con barras de error (yerr) para mostrar variabilidad entre folds
# capsize: Tamaño de las "tapas" en las barras de error
plt.bar(nombres, rmse_values, yerr=std_values, capsize=5, alpha=0.7, color='lightcoral')
plt.title('RMSE con Validación Cruzada')
plt.ylabel('RMSE')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
# Scores negativos porque sklearn maximiza, pero MSE se minimiza
plt.bar(nombres, [cv_scores[n]['mean_score'] for n in nombres],
        yerr=[cv_scores[n]['std_score'] for n in nombres],
        capsize=5, alpha=0.7, color='lightblue')
plt.title('Score Promedio (Negativo MSE)')
plt.ylabel('Score')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()



In [ ]:
scores


---

## Resumen: Caso 1 - Calibración de Sensores

### Problema de Ingeniería Electrónica
- **Sensor de temperatura con desgaste** → Relación voltaje-temperatura distorsionada
- **Solución**: Modelo de regresión lineal para calibración

### Resultados Obtenidos
- **Ecuación**: `T = -1.26 + 21.18 × V`
- **RMSE**: 0.60°C (excelente precisión)
- **R²**: 0.9997 (99.97% de varianza explicada)

### Aplicaciones Prácticas
- Sistemas de control de temperatura
- Instrumentación industrial
- Monitoreo ambiental

---


## Resumen: Caso 2 - Fabricación de Resistencias

### Problema de Manufactura Compleja
- **4 Variables de proceso**: Temperatura, tiempo, espesor, dopante
- **Objetivo**: Optimizar resistencia del componente

### Comparación de Modelos

| Modelo              | RMSE       | R²         | Características        |
|---------------------|-----------:|-----------:|------------------------|
| Regresión Lineal    | 0.0005     |     0.7478 | Simple, interpretable  |
| Ridge (α=0.1)       | 0.0005     |     0.7479 | Regularización L2      |
| Lasso (α=0.01)      | 0.0009     |     -0.0859| Selección de variables |
| **Random Forest**   | **0.0004** | **0.8202** | **Mejor rendimiento**  |

### Lecciones Aprendidas
- **Estandarización crucial** para modelos lineales
- **Random Forest** maneja relaciones no lineales
- **Lasso** puede fallar con múltiples variables importantes

---


## Técnicas Avanzadas: Validación Cruzada y Grid Search

### ¿Por qué Validación Cruzada?
- **Problema**: Una sola división train/test puede ser sesgada
- **Solución**: K-Fold CV divide datos en K partes, entrena K veces
- **Beneficio**: Estimación más robusta del rendimiento

### Grid Search con CV
```python
param_grid = {'alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]}
grid_search = GridSearchCV(Ridge(), param_grid, cv=5)
grid_search.fit(X_train_scaled, y_train)
```

### Resultados de CV (5-fold)
- **Linear Regression**: RMSE = 0.0006 ± 0.0004
- **Ridge (optimizado)**: RMSE = 0.0006 ± 0.0005  
- **Random Forest**: RMSE = 0.0006 ± 0.0005

### Interpretación
- **±** indica **variabilidad** entre folds (menor = más estable)
- **Todos los modelos** muestran rendimiento similar y estable

---


## Métricas de Evaluación en Regresión

### Principales Métricas Utilizadas

#### 1. **RMSE (Root Mean Square Error)**

- **Fórmula**: $\sqrt{\sum (y_\textsf{real} - y_\textsf{pred})^2 / n}$
- **Ventajas**: Mismas unidades que variable objetivo, penaliza errores grandes
- **Interpretación**: Menor = mejor

#### 2. **R² (Coeficiente de Determinación)**

- **Rango**: 0 a 1 (puede ser negativo si modelo es muy malo)
- **Interpretación**: % de varianza explicada por el modelo
- **R² = 0.82** → Modelo explica 82% de la variabilidad

#### 3. **MAE (Mean Absolute Error)**

- **Fórmula**: $\sum |y_\textsf{real} - y_\textsf{pred}| / n$
- **Ventaja**: Menos sensible a outliers que RMSE
- **Uso**: Complementa RMSE para análisis completo

### Análisis de Residuos

- **Residuos**: $y_\textsf{real} - y_\textsf{pred}$
- **Ideal**: Distribuidos aleatoriamente alrededor de 0
- **Problemas**: Patrones indican falta de ajuste del modelo

---


## Comparación de Algoritmos de Regresión

### **Linear Regression**
- ✓ Simple e interpretable
- ✓ Rápido de entrenar
- ✕ Asume relación lineal
- ✕ Sensible a outliers y multicolinealidad

### **Ridge Regression (L2)**
- ✓ Controla sobreajuste con regularización  
- ✓ Maneja multicolinealidad
- ✓ Todos los coeficientes se mantienen (≠ 0)
- ✕ No elimina variables irrelevantes

### **Lasso Regression (L1)**  
- ✓ Selección automática de variables (coef → 0)
- ✓ Produce modelos más simples
- ✕ Puede eliminar variables importantes
- ✕ Inestable con variables correlacionadas

### **Random Forest**
- ✓ Maneja relaciones no lineales
- ✓ Robusto a outliers
- ✓ No requiere estandarización
- ✕ Menos interpretable ("caja negra")

---


## Conclusiones y Recomendaciones

### 🎯 **Principales Aprendizajes**

1. **Preprocesamiento es crítico**
   - Estandarización obligatoria para Ridge/Lasso
   - Análisis exploratorio revela patrones importantes

2. **No hay modelo universalmente mejor**
   - Linear: Simplicidad e interpretabilidad
   - Random Forest: Mejor rendimiento en datos complejos  
   - Ridge: Equilibrio entre sesgo y varianza

3. **Validación cruzada es esencial**
   - Una sola división train/test puede engañar
   - Grid Search automatiza optimización de hiperparámetros

### 💡 **Recomendaciones Prácticas**

- **Empezar simple**: Linear Regression como baseline
- **Probar regularización**: Ridge si hay multicolinealidad  
- **Considerar ensemble**: Random Forest para relaciones complejas
- **Siempre validar**: Cross-validation + métricas múltiples
- **Interpretar residuos**: Detecta problemas del modelo

### 🚀 **Próximos Pasos**
- Explorar otros algoritmos (SVM, Neural Networks)
- Técnicas de feature engineering más avanzadas
- Optimización bayesiana de hiperparámetros

---

### ¡Gracias por su atención!
### ¿Preguntas?
